In [1]:
import os
import googlemaps
from datetime import datetime, date
import responses
import numpy as np

gmaps_key = os.environ["GMAPS_API_KEY"]
gmaps = googlemaps.Client(key=gmaps_key)

At the beginning we should approximate the amount of time people statistically spend in chosen category of places.
We also should determine approximate hours appropriate to visit such a place, but later we will also verify opening hours.
General assumptions about each category of sites.

- CATEGORY : TIMESLOT : AVERAGE TIME
- accommodation - we can suggest some places (taking into consideration budget, distance to the city centre and ratings)
- car_services: IGNORE (but car mode with suggesting car plot can be considered)
- christianity: IGNORE - can be suggested in other section
- culture: 9 - 19 : 2h
- family_entertainment: 8 - 20 : 3h
- food_and_drink: 7 - 24: 2h
- hinduism: IGNORE - can be suggested in other section
- islam: IGNORE - can be suggested in other section
- judaism: IGNORE - can be suggested in other section
- nightlife: 21 - 6 : 2h
- shopping: 9 - 20: 2h - shopping mall to be considered
- sports and fitness: 8 - 22 : 3h
- travel services: IGNORE
- welness: 8 - 22 - 2h


In [44]:
import json
cat_json = {}
with open("..\json\categories.json") as f:
    cat_json = json.load(f)

class Category:
    def __init__(self, name:str, categories_list:list, ignore:bool, start_time:int = None, end_time:int = None, duration:int = None):
        self.name = name
        self.ignore = ignore
        self.start_time = start_time
        self.end_time = end_time
        self.duration = duration
        self.categories_list = categories_list
        self.probability: int = None
        # self.available_hours = None
    
    def set_probability(self, prob):
        self.probability = prob

    # def set_available_hours(self, hours_number):
    #     self.available_hours = hours_number
        
accommodation = Category("accommodation", ["lodging", "rv_park", "campground"], True)
car_services = Category("car_services", cat_json["car_services"], True)
christianity = Category("christianity", cat_json["christianity"], True)
culture = Category("culture", cat_json["culture"], False, 9, 19, 2)
family_entertainment = Category("family_entertainment", cat_json["family_entertainment"], False, 8, 20, 3)
food_and_drink = Category("food_and_drink", cat_json["food_and_drink"], False, 7, 24, 2)
hinduism = Category("hinduism", cat_json["hinduism"], True)
islam = Category("islam", cat_json["islam"], True)
judaism = Category("judaism", cat_json["judaism"], True)
nightlife = Category("nightlife", cat_json["nightlife"], False, 21, 6, 2)
shopping = Category("shopping", cat_json["shopping"], True)
sports_and_fitness = Category("sports_and_fitness", cat_json["sports_and_fitness"], False, 8, 22, 3)
travel_services = Category("travel_services", cat_json["travel_services"], True)
wellness = Category("wellness", cat_json["wellness"], False, 8, 22, 2)

cheap_travel = cat_json["cheap_transport"]
expensive_travel = cat_json["expensive_transport"]

categories_list = [accommodation, car_services, christianity, culture, family_entertainment, food_and_drink, hinduism, islam, judaism, nightlife, shopping, sports_and_fitness, travel_services, wellness]

In [66]:
from enum import Enum
import math
import requests
class TravelWith(Enum):
    alone = 1
    with_partner = 2
    with_family = 3

user_preferences = {'nightlife': 0.13586715459823606, 'christianity': 0.2641089591715071, 'judaism': 0.23523988525072737, 'hinduism': 0.12414621661106745, 'islam': 0.0, 'car_services': 0.17760687271753947, 'culture': 0.31209163031092396, 'food_and_drink': 0.23561079038514032, 'sports_and_fitness': 0.06975500285625458, 'wellness': 0.233438245455424, 'family_entertainment': 0.2411984403928121, 'shopping': 0.0, 'travel_services': 0.21448883761962256}

# #It is useful or should be removed?
# for i in categories_list:
#     for key, value in user_preferences:
#         if i.name == key:
#             i.set_probability(value)

user_destination = 'Cracow'
user_start_datatime = datetime(2023,6, 1, 13)
user_end_datatime = datetime(2023,6, 4, 17)
public_transport_accept = True
bicycle_travel_accept = False
car_travel_accept = True
cost_rate = 4 #scale from 0 to 4
travel_with = TravelWith.alone

class User:
    def __init__(self, preferences:dict, destination:str, start_time:datetime, end_time:datetime, public_transport_accept:bool, bicycle_travel_accept:bool, car_travel_accept:bool, cost_rate:int, travel_with:TravelWith):
        self.preferences = preferences
        self.destination = destination
        self.start_time = start_time
        self.end_time = end_time
        self.public_transport_accept = public_transport_accept
        self.bicycle_travel_accept = bicycle_travel_accept
        self.car_travel_accept = car_travel_accept
        self.cost_rate = cost_rate
        self.travel_with = travel_with
        self.available_days = []
        self.main_preferences = {}
        self.religion = {}
        self.sorted_categories = []
        self.num_of_places_for_main_categories = []
        self.num_of_places_for_main_categories_for_search = []
        self.prepare_available_days()
        self.get_main_preferences()
        self.get_religion()
        self.get_the_most_interesting_categories()
        self.get_sorted_cat_list()
        self.get_number_of_places_from_each_category_to_see()
        self.get_number_of_places_to_be_found()
        self.prepare_city_coordinates()
        self.prepare_search_radius()
        self.prepare_places_list()

    def prepare_available_days(self):
        days_in_city = self.end_time.day - self.start_time.day + 1
        prob_values_mean =  np.mean(list(self.preferences.values()))
        prob_values_median =  np.median(list(self.preferences.values()))

        #Here we define time slots for city exploration taking into account user nightlife's preference
        if self.preferences["nightlife"] > 1.5 * prob_values_mean and self.preferences["nightlife"]>=prob_values_median:
            start_hour = 9
            end_hour = 3
        elif self.preferences["nightlife"] <= 1.5 * prob_values_mean and self.preferences["nightlife"] >= prob_values_mean or self.preferences["nightlife"] > 1.5 * prob_values_mean and self.preferences["nightlife"]<prob_values_median:
            start_hour = 8
            end_hour = 24
        else:
            start_hour = 7
            end_hour = 23
        
        for i in range (days_in_city):
            if i == 0:
                if self.start_time.hour > start_hour:
                    curr_day = CityDay(self.start_time.hour, end_hour)
                else: 
                    curr_day = CityDay(start_hour, end_hour)
            elif i == days_in_city - 1:
                if self.end_time.hour > end_hour:
                    curr_day = CityDay(start_hour, end_hour)
                else: 
                    curr_day = CityDay(start_hour, self.end_time.hour)
            else:
                curr_day = CityDay(start_hour, end_hour)
                    
            self.available_days.append(curr_day)

    def get_main_preferences(self):
        remove_keys = ['christianity', 'judaism', 'hinduism', 'islam', 'car_services', 'travel_sevices']
        self.main_preferences = {key: value for key, value in self.preferences.items() if key not in remove_keys}
            
    def get_religion(self):
        #if one has really bigger prob than others take the one which user is the most interested in, else check if probabilities are similiar to each other and relatively big, then propose some religion places to see, else if one is bigger and relatively big then propose one temple
        religion_keys = ['christianity', 'judaism', 'hinduism', 'islam']
        religions = {key: value for key, value in self.preferences.items() if key in religion_keys}
        sorted_list = sorted(religions.items(), key=lambda x: x[1], reverse=True)
        sorted_religion = {item[0]: item[1] for item in sorted_list}
        
        iter_ = iter(sorted_religion.items())
        first_item = next(iter_)
        second_item = next(iter_)
        third_item = next(iter_)
        fourth_item = next(iter_)
        
        if first_item[1] > 1.5 * second_item[1] and first_item[1] > 0.2:
            religion = first_item
            self.religion = {item[0]: item[1] for item in religion}
        elif second_item[1] > 1.5 * third_item[1] and second_item[1] > 0.2:
            religion = first_item, second_item
            self.religion = {item[0]: item[1] for item in religion}
        elif fourth_item[1] > 0.2:
            self.religion = sorted_religion
             
    def get_the_most_interesting_categories(self):
        self.sorted_preferences = sorted(self.main_preferences.items(), key=lambda x: x[1], reverse=True)
        if(self.available_days.count == 2):
            if("nightlife" in key for key,value in self.sorted_preferences[:3]):
                self.main_preferences = dict(self.sorted_preferences[:4])
            else:
                self.main_preferences = dict(self.sorted_preferences[:3])
        elif(self.available_days.count == 3):
            self.main_preferences = dict(self.sorted_preferences[:4])
        elif(self.available_days.count == 4):
            self.main_preferences = dict(self.sorted_preferences[:5])
        elif(self.available_days.count == 5 or self.available_days.count == 6):
            self.main_preferences = dict(self.sorted_preferences[:6])
        else:
            self.main_preferences = self.sorted_preferences
            
    def get_prob_sum(self):
        prob_sum = 0
        for key,value in self.main_preferences:
            prob_sum += value
        return prob_sum
    
    def get_sorted_cat_list(self):
        for key,value in self.main_preferences:
            for cat in categories_list:
                if(key == cat.name):
                    self.sorted_categories.append(cat)
    
    def get_number_of_places_from_each_category_to_see(self):
        available_hours_numbers = []
        number_of_places_to_see = []
        num_of_places = 0
        for key,value in self.main_preferences:
            for cat in categories_list:
                if(cat.ignore == False):
                    if(key == cat.name):
                        hours_num = 0
                        for day in self.available_days:
                            overlap_start = max(cat.start_time, day.start_hour)
                            overlap_end = min(cat.end_time, day.end_hour)
                            hours_num += overlap_end-overlap_start
                        available_hours_numbers.append(hours_num)
                        number_of_places_to_see.append(hours_num/cat.duration)
                    
        num_of_places = (sum(number_of_places_to_see) / len(self.main_preferences)) 
        prob_sum = self.get_prob_sum()
        
        for key,value in self.main_preferences:
            print((value/prob_sum) * num_of_places)
            self.num_of_places_for_main_categories.append(math.ceil(value/prob_sum * num_of_places))
            
    def get_number_of_places_to_be_found(self):
        for num in self.num_of_places_for_main_categories:
            self.num_of_places_for_main_categories_for_search.append(math.ceil(1.8 * num))
            
    def prepare_city_coordinates(self):
        endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'
        response = requests.get(endpoint, params={'address': self.destination, 'key': gmaps_key})
        json_response = response.json()
        self.lat = json_response['results'][0]['geometry']['location']['lat']
        self.lng = json_response['results'][0]['geometry']['location']['lng']
    
    def prepare_search_radius(self):
        if self.bicycle_travel_accept is False and self.public_transport_accept is False and self.car_travel_accept:
            self.radius = 4000
        if self.public_transport_accept is True :
            self.radius = 8000
        if self.bicycle_travel_accept is True:
            self.radius = 10000
        if self.car_travel_accept is True:
            self.radius = 18000
    
    def prepare_places_list(self):
        i = 0
        for cat in self.sorted_categories:
            curr_cat_places_list = []
            for tp in cat.categories_list:
                search_req_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={self.lat}%2C{self.lng}&radius={self.radius}&type={tp}&maxprice={self.cost_rate}&key={gmaps_key}"

                payload={}
                headers = {}
                response = requests.request("GET", search_req_url, headers=headers, data=payload)
                
                json_response = response.json()
                results = json_response['results']
                
                curr_cat_places_list += results
            
            print(f"curr_cat_list {len(curr_cat_places_list)}")
            
            if(len(curr_cat_places_list) < self.num_of_places_for_main_categories[i]):
                temp = self.num_of_places_for_main_categories[i]
                self.num_of_places_for_main_categories[i] = len(curr_cat_places_list)
                if(len(self.sorted_categories) > i+1):
                    self.num_of_places_for_main_categories[i+1] += (temp - len(curr_cat_places_list))
                    self.num_of_places_for_main_categories_for_search[i+1] += (temp - len(curr_cat_places_list))
                else:
                    cat_to_be_added = self.sorted_preferences[i+1][0]
                    for cat in categories_list:
                        if(cat.name == cat_to_be_added):
                            self.sorted_categories.append(cat)
                            self.num_of_places_for_main_categories.append(self.num_of_places_for_main_categories[i] - len(curr_cat_places_list))
                            self.num_of_places_for_main_categories_for_search.append(math.ceil((self.num_of_places_for_main_categories[i] - len(curr_cat_places_list))*1.8))
            i += 1
            
            #take appropriate number of places from each category (at least expected number if search number is not possible) and then clasterize itd.
            
            print(cat.name)
            print(len(curr_cat_places_list))
            print(self.sorted_categories)
                

    def check_alg(self):
        print(self.sorted_categories)
        print(self.num_of_places_for_main_categories)
        print(self.num_of_places_for_main_categories_for_search)
        
        #weź ze wszystkich dni dostępną ilość godzin w slotach dla każdej kategorii -> każda ma jakąś ilość godzin
        
        #współczynik prawdo/suma_prawdo -> zakres / średni czas na miejsce (ilość możliwych do zobaczenia)
        
        #SUMA (zakres / średni czas na miejsce (ilość możliwych do zobaczenia)) / ilość kategorii * współczynik prawdo/suma_prawdo
        
        #Ilość miejsc * 1,5 przed klastrowaniem w traski -> rating miejsca(1-5) * 0.7 + [jak tańsze to 2 pkt -> jak droższe o 1 -> 1.5 jak droższe o 2 -> 0.5] -> jako metryke odległości to po prostu odległość - (znormalizować jakoś) (tu zależnie od środka lokomocji * )
        # Po tym zostanie nam 1,2 tych miejsc
        
        # podziel wszystko na ilość klastrów równą ilości dni 
        
        # walidacja - czy otwarty w godzinach odwiedzenia -> jak nie to szacher macher (może genetyczny na podmianki)
        
        #wypluć punkty
   
class CityDay:
    def __init__(self, start_hour:int, end_hour:int):
        self.start_hour = start_hour
        self.end_hour = end_hour
        self.places_list = []
    
    def add_place(self, place):
        self.places_to_see.append(place)


user = User(user_preferences, user_destination, user_start_datatime, user_end_datatime, public_transport_accept, bicycle_travel_accept, car_travel_accept, cost_rate, travel_with)
user.check_alg()


1.7489275195836218
1.351649801256697
1.3203372189279656
1.3081625136616715
1.2019720951271227
0.7613847428322403
0.39089944194401427
0.0
curr_cat_list 3
culture
3
[<__main__.Category object at 0x000001F022346C70>, <__main__.Category object at 0x000001F022346CA0>, <__main__.Category object at 0x000001F02225EA30>, <__main__.Category object at 0x000001F02225E100>, <__main__.Category object at 0x000001F02225E310>, <__main__.Category object at 0x000001F02225E970>, <__main__.Category object at 0x000001F02225E1F0>, <__main__.Category object at 0x000001F02225E130>]
curr_cat_list 0
family_entertainment
0
[<__main__.Category object at 0x000001F022346C70>, <__main__.Category object at 0x000001F022346CA0>, <__main__.Category object at 0x000001F02225EA30>, <__main__.Category object at 0x000001F02225E100>, <__main__.Category object at 0x000001F02225E310>, <__main__.Category object at 0x000001F02225E970>, <__main__.Category object at 0x000001F02225E1F0>, <__main__.Category object at 0x000001F02225E13

In [22]:
import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522%2C151.1957362&radius=1500&type=restaurant&keyword=cruise&key={gmaps_key}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{
   "html_attributions" : [],
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : -33.8675219,
               "lng" : 151.2016502
            },
            "viewport" : {
               "northeast" : {
                  "lat" : -33.86631932010728,
                  "lng" : 151.2031278298927
               },
               "southwest" : {
                  "lat" : -33.86901897989272,
                  "lng" : 151.2004281701072
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png",
         "icon_background_color" : "#7B9EB0",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet",
         "name" : "Sydney Harbour Dinner Cruises",
         "opening_hours" : {
            "open_now" : false
         },
         "photos" : [
            {
               "heig

In [ ]:
#ALGORITHM

Input:
- categories_probs: a dictionary with keys as place categories and values as probabilities of interest and visiting sequence
- start_date: start date of tour
- end_date: end date of tour
- transportation_mode: preferred mode of transportation (walking, bicycle, car)

Output:
- A list of tours, one for each day of the tour period, with each tour containing a list of recommended places to visit

1. Calculate the total number of places to recommend based on the length of the tour and the preferred transportation mode
   1.1. Define a maximum distance to travel between places based on the transportation mode
   1.2  Define the maximum number of hours a user is willing to spend traveling in a day
   1.3. Calculate the maximum number of places that can be visited based on the tour length and the maximum distance and time needed to spend in this places
   1.4. Multiply the maximum number of places by the probabilities of interest for each category to determine how many places to recommend from each category

2. Group the recommended places based on their proximity to each other (taking into consideration geographic coordinates)
   2.1. Calculate the distance between each pair of places
   2.2. Group places that are within the maximum distance of each other into clusters
   2.3. For each cluster, recommend the place with the highest rating as the primary destination and the others as alternative options

3. For each day of the tour, recommend a tour based on the recommended places and their opening hours
   3.1. Sort the recommended places by their rating and opening hours
   3.2. Starting from the earliest opening hour, select the next available place that has not been visited yet and add it to the tour
   3.3. If there are no more places available, end the tour for the day

4. Return the list of tours, each containing the recommended places for each day